In [ ]:
import os
OPENAI_API_TOKEN = "ss"
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [38]:
prompt = """
markdow
here is the story:
{story}
**Role**: You are an advanced text analysis expert.

**Task**: Extract and describe the main character from the provided story.(keyword only format) Limit the description to one main character. Ensure comprehensive details are included as follows:

1. **Character's Name**: Clearly state the character's name. Example: "Ella".
2. **Character's Age**: Specify the character's age. Example: "8 years old".
3. **Character's Gender**: Identify the character's gender. Example: "Female".
4. **Character's Occupation**: Describe the character's occupation. Example: "Student".
5. **Outfit Description**: Provide a detailed description of the character's outfit, including elements such as:
   - Top
   - Bottom
   - Shoes
   - Accessories
"""

response_format={
    "name": "story_dict",
    "schema": {
        "type": "object",
        "properties": {
            "content": {
                "name": "name of the character",
                "Occupation":"Describe the character's occupation. Example: 'Student'",
                "gender": "male or female",
                "Age": "estimate age of the character",
                "Outfit_description": {
                    "top": {
                        "description": "Describe the upper body outfit such as shirt, hat, glasses etc.",
                        "type": "string"
                    },
                    "bottom": {
                        "description": "describe the lower body outfit such as shoes, skirt, jeans, etc.",
                        "type": "string"
                    },
                    "accessory": {
                        "description": "descripe any accessroy character may have.",
                        "type": "string"
                    }
                },
                "required": ["name", "Occupation", "gender", "Age", "Outfit_description"],
                "additionalProperties": False 
            }
        },
        "additionalProperties": False
    }
}


following is the summary

In [35]:
input = 7
prompt = """
```markdown
**Story Input**: 
```
{story}

**Role**: You are an advanced text analysis expert specializing in generating vivid, action-oriented descriptions.

**Task**: Extract and describe the main character and their environment from the provided story. Your analysis should focus on creating detailed, image-inspiring action scenes using descriptive keywords. Follow the structured outputs below:

### Action-Oriented Sentence Summary:
- Provide a concise summary of the story, focusing on dynamic actions, emotions, and vivid settings.
- Highlight only one key scene from the entire paragraph.
- Keep the output short, avoiding extra actions or background.
- Use active language to describe key moments, concentrating on what the main character is doing and experiencing.
- Emphasize the scene's background and the main character's actions.
- Use simple language. Example: "A figure strides to a lake embraced by towering trees."

### Background Details:
- Describe any background elements mentioned, using complete sentences with detailed descriptions. Example: "Lush green trees with sturdy brown trunks."

### Descriptive Paragraph Structure:
- Divide the content into exactly {input} descriptive paragraphs.
- Each paragraph should vividly portray a single scene, detailing the main character's actions, appearance, surroundings, and emotional tone.
- Ensure equal emphasis on each element.
...
- Avoid using character names; instead, describe the main character's outfit and appearance.

**Objective**: Generate detailed, action-focused descriptions for vivid and accurate image generation.
```
"""
content = {}
for i in range(input): 
    storyNum = f"Summary of paragraph {i + 1}" 
    content[f"paragraph {i + 1}"] = storyNum

response_format = {
    "name": "story_dict",
    "schema": {
        "name": "Story Content Schema",
        "type": "object",
        "properties": {
            "content": {
                "type": "object",
                "properties": {key: {"type": "string", "description": value} for key, value in content.items()},
                "required": ["type"],
                "additionalProperties": False
            }
        },
        "additionalProperties": False
    }
}



print(response_format)


{'name': 'story_dict', 'schema': {'name': 'Story Content Schema', 'type': 'object', 'properties': {'content': {'type': 'object', 'properties': {'paragraph 1': {'type': 'string', 'description': 'Summary of paragraph 1'}, 'paragraph 2': {'type': 'string', 'description': 'Summary of paragraph 2'}, 'paragraph 3': {'type': 'string', 'description': 'Summary of paragraph 3'}, 'paragraph 4': {'type': 'string', 'description': 'Summary of paragraph 4'}, 'paragraph 5': {'type': 'string', 'description': 'Summary of paragraph 5'}, 'paragraph 6': {'type': 'string', 'description': 'Summary of paragraph 6'}, 'paragraph 7': {'type': 'string', 'description': 'Summary of paragraph 7'}}, 'required': ['type'], 'additionalProperties': False}}, 'additionalProperties': False}}


now is the code

In [ ]:
from POP import PromptFunction
import json

story = """
"In the heart of Willow Wood, where the sun painted the forest in warm hues, lived a kind-hearted boy named Oliver. With curly brown hair and eyes like two curious marbles, he loved nothing more than spending his days among the animals that called the lush forest their home. Oliver had a special gift; he could talk to animals, sharing secrets and laughter with the creatures who felt more like family than friends."
(string) "One sparkling morning, as Oliver skipped along the forest path, he found Sally Squirrel in distress. "What's wrong, Sally?" Oliver asked, his voice full of concern. "Oh, Oliver," Sally chattered nervously, "Old Oak's grand acorn treasure has vanished! He's so heartbroken, and the forest just doesn't feel the same without it." Oliver promised he would help find the treasure, his heart filling with the determination that only a true friend can muster.""Gathering his friends, Oliver set out on a journey deeper into the magical forest than he had ever dared venture. Sammy the wise owl, Benny the brave little badger, and Lila the swift fox joined the quest. Together, they formed a team, each bringing special skills to solve the mystery. The forest was full of surprises, from sparkling streams to towering trees that whispered in the breeze."
"As they traversed through dense thickets and mossy trails, they found clues hidden in unexpected places. Lila's sharp eyes spotted a shining feather stuck in a bush, while Sammy unfolded a map using his clever intuition. "This feather belongs to Ruby Raven," Sammy hooted excitedly. Picking up the pace, they followed a shimmering path that seemed to glow just for them."
(string) "When night fell, casting shadows that danced under the silver moon, fear tried to creep into their hearts. But Oliver, with bravery in his veins, reminded them of their mission. "Remember, we're in this together," he said, warmly holding their spirits high. It was these words that kept their courage bright in the darkness, where mysterious sounds lurked behind the trees."
(string) Finally, as dawn broke and bathed the forest in golden light, they reached a secluded glen known to be Ruby's secret hideaway. There, Ruby Raven perched proudly on a bough, her beady eyes glinting with mischief. "Why have you come, little ones?" she croaked, unfurling her glossy wings. Oliver stepped forward, his voice steady but kind. "We only ask you to return Old Oak's acorn treasure so the forest can be joyful once more.""Ruby hesitated, her heart touched by Oliver's earnest plea. "I only borrowed it, wanting to add a bit of sparkle to my nest," she explained, a touch of guilt in her voice. Understanding her desire for beauty, Oliver proposed a fair trade. "Help us find other treasures, Ruby, and we'll all share in the forest's wonders." So touched by their teamwork, she promised to guide them."
"""

prompt = """
markdow
here is the story:
{story}
**Role**: You are an advanced text analysis expert.

**Task**: Extract and describe the main character from the provided story.(keyword only format) Limit the description to one main character. Ensure comprehensive details are included as follows:

1. **Character's Name**: Clearly state the character's name. Example: "Ella".
2. **Character's Age**: Specify the character's age. Example: "8 years old".
3. **Character's Gender**: Identify the character's gender. Example: "Female".
4. **Character's Occupation**: Describe the character's occupation. Example: "Student".
5. **Outfit Description**: Provide a detailed description of the character's outfit, including elements such as:
   - Top
   - Bottom
   - Shoes
   - Accessories
"""

response_format={
    "name": "story_dict",
    "schema": {
        "type": "object",
        "properties": {
            "content": {
                "name": "name of the character",
                "Occupation":"Describe the character's occupation. Example: 'Student', if not human, describe the race",
                "gender": "male or female",
                "Age": "estimate age of the character",
                "Outfit_description": {
                    "top": {
                        "description": "Describe the upper body outfit such as shirt, hat, glasses etc.",
                        "type": "string"
                    },
                    "bottom": {
                        "description": "describe the lower body outfit such as shoes, skirt, jeans, etc.",
                        "type": "string"
                    },
                    "accessory": {
                        "description": "descripe any accessroy character may have.",
                        "type": "string"
                    }
                },
                "required": ["name", "Occupation", "gender", "Age", "Outfit_description"],
                "additionalProperties": False 
            }
        },
        "additionalProperties": False
    }
}

pf = PromptFunction(prompt)

pf.set_temperature(0.5)
response = pf.execute(story = story, RESPONSE_FORMAT=response_format)

json.loads(response)
output_file = "output.json"
with open(output_file, 'w') as file:
    json.dump(json.loads(response), file, indent=4)
    
input = 6
prompt = """
```markdown
**Story Input**:  
{story}

**Role**: As a creative text analysis expert, your task is to generate concise and visually clear descriptions.

**Goal**: Create short, vivid paragraphs that describe distinct, self-contained events or scenes from the story. Each paragraph should be independent, focusing on a single key action or setting.

**Guidelines for Output**:

### Paragraph Structure:

- **Single Event Focus**: Each paragraph should capture one main event or scene.
  - **Example**: "A dragon slides down a colorful water park slide."
  - **Avoid**: Long backstories, complex contexts, or additional details.

- **Short and Simple**: Sentences should be brief and straightforward.
  - **Good**: "A mouse eats a piece of cheese on a wooden table."
  - **Bad**: "A mysterious mouse sits in a witch's house, savoring a piece of cheese while planning its next move."

- **Independent Descriptions**: Each paragraph must stand alone, without relying on previous paragraphs for context.
  - **Good**: "A dragon walks through a grassy field."
  - **Bad**: "It then flies over the mountains." (Unclear what "it" refers to.)

### Character and Setting Description:

- Use vivid but simple details to describe the scene.
- Avoid overly elaborate language or unnecessary adjectives.
  - **Good**: "A squirrel climbs a tall oak tree."
  - **Bad**: "A nimble squirrel scampers up a towering oak tree, its branches swaying in the cool autumn breeze."

### What to Avoid:

- Do not refer back to previous paragraphs.
- Do not describe complex timelines or unrelated subplots.
- Do not use name in the paragraph, instead use clear describtion of that character.
- Keep characters and settings straightforward without adding unnecessary context.

**Output Objective**: Produce independent, short paragraphs describing a single event, character, or setting per paragraph. The aim is to provide clear and vivid inputs for an image generator to create accurate visuals.
```
"""
content = {}
for i in range(input): 
    storyNum = f"Summary of paragraph {i + 1}" 
    content[f"paragraph {i + 1}"] = storyNum

response_format = {
    "name": "story_dict",
    "schema": {
        "name": "Story Content Schema",
        "type": "object",
        "properties": {
            "content": {
                "type": "object",
                "properties": {key: {"type": "string", "description": value} for key, value in content.items()},
                "required": ["type"],
                "additionalProperties": False
            }
        },
        "additionalProperties": False
    }
}


pf = PromptFunction(prompt)
pf.set_temperature(0.5)

response = pf.execute(story = story, RESPONSE_FORMAT=response_format)


json.loads(response)
output_file = "output_2.json"

with open(output_file, 'w') as file:
    json.dump(json.loads(response), file, indent=4)
    
file1 = "output.json"
file2 = "output_2.json"
output_file = "combined.json"

with open(file1, "r") as f1:
    json1 = json.load(f1)

with open(file2, "r") as f2:
    json2 = json.load(f2)

combined_json = {"content": {**json1["content"], **json2["content"]}}

output_file = "final_output.json"
with open(output_file, "w") as output:
    json.dump(combined_json, output, indent=4)




In [55]:
import json

json.loads(response)
output_file = "output.json"
with open(output_file, 'w') as file:
    json.dump(json.loads(response), file, indent=4)
    


In [45]:
prompt = """
**Story Input**:  
{story}

Role: You are a creative text analysis expert tasked with generating concise and visually clear descriptions.

Goal: Generate short, vivid paragraphs describing distinct, self-contained events or scenes from the story. Each paragraph must be independent, focusing on one key action or setting at a time.
Guidelines for Output:
Paragraph Structure:

    Single Event Focus: Each paragraph should describe one main event or scene.
        Example: "A dragon slides down a colorful water park slide."
        Avoid long backstories, complex contexts, or additional details.
    Short and Simple: Sentences should be brief and straightforward.
        Good: "A mouse eats a piece of cheese on a wooden table."
        Bad: "A mysterious mouse sits in a witch's house, savoring a piece of cheese while planning its next move."
    Independent Descriptions: Each paragraph must stand alone without relying on previous paragraphs for context.
        Good: "A dragon walks through a grassy field."
        Bad: "It then flies over the mountains." (Unclear what "it" refers to.)

Character and Setting Description:

    Describe the scene using vivid but simple details.
    Avoid overly elaborate language or unnecessary adjectives.
        Good: "A squirrel climbs a tall oak tree."
        Bad: "A nimble squirrel scampers up a towering oak tree, its branches swaying in the cool autumn breeze."

What to Avoid:

    Do not refer back to previous paragraphs.
    Do not describe complex timelines or unrelated subplots.
    Keep characters and settings straightforward without adding unnecessary context.

Output Objective: Produce independent, short paragraphs describing a single event, character, or setting per paragraph. The goal is to provide clear and vivid inputs for an image generator to create accurate visuals.
"""


In [46]:
use_prompt = "fabric"
pf = PromptFunction(prompt)
print(pf._improve_prompt(use_prompt=use_prompt))

```markdown
**Story Input**:  
{story}

**Role**: As a creative text analysis expert, your task is to generate concise and visually clear descriptions.

**Goal**: Create short, vivid paragraphs that describe distinct, self-contained events or scenes from the story. Each paragraph should be independent, focusing on a single key action or setting.

**Guidelines for Output**:

### Paragraph Structure:

- **Single Event Focus**: Each paragraph should capture one main event or scene.
  - **Example**: "A dragon slides down a colorful water park slide."
  - **Avoid**: Long backstories, complex contexts, or additional details.

- **Short and Simple**: Sentences should be brief and straightforward.
  - **Good**: "A mouse eats a piece of cheese on a wooden table."
  - **Bad**: "A mysterious mouse sits in a witch's house, savoring a piece of cheese while planning its next move."

- **Independent Descriptions**: Each paragraph must stand alone, without relying on previous paragraphs for context.
 

In [81]:
from POP import PromptFunction
prompt = """markdown

here is the story:
{story}
**Role**: You are an advanced text analysis expert.

**Task**: Extract and describe the main character from the provided story.(keyword only format) Limit the description to one main character. Ensure comprehensive details are included as follows:

1. **Character's Name**: Clearly state the character's name. Example: "Ella".
2. **Character's Age**: Specify the character's age. Example: "8 years old".
3. **Character's Gender**: Identify the character's gender. Example: "Female".
4. **Character's Occupation**: Describe the character's occupation. Example: "Student".
5. **Outfit Description**: Provide a detailed description of the character's outfit, including elements such as:
   - Top
   - Bottom
   - Shoes
   - Accessories
"""
                
story = """In the quiet village of Everturn, nestled between sprawling green hills and a shimmering silver lake, there lived an old clockmaker named Elias. Known for his intricate creations, Elias could repair any timepiece brought to his shop. But his true pride lay in the masterpiece that hung on the wall of his workshop—a towering grandfather clock unlike any other. It was said to have the power to glimpse moments yet to come.

The villagers often whispered about the clock’s origins. Some believed Elias crafted it with the help of enchanted tools passed down from ancient times. Others thought he had made a deal with a spirit of time itself. Whatever the truth, the clock was more than a mere instrument—it was a legacy.

Elias had no children, but he had taken on a young apprentice named Lena. Bright-eyed and curious, Lena had a knack for repairing delicate mechanisms. She admired Elias, not just for his skill but for his kindness. He would never turn away a customer, even if they couldn’t pay. “Time,” he often said, “is a gift, not a currency.”

One stormy evening, as lightning danced across the sky, a stranger arrived in Everturn. Cloaked in shadows, he stepped into Elias’s workshop, his presence unsettling.

“I have a request,” the man said, his voice low and gravelly. He produced a pocket watch from his cloak, its surface adorned with strange, swirling patterns. “Fix this, and I’ll pay you with something greater than gold.”

Elias took the watch and studied it. Its mechanisms were like nothing he had ever seen, gears turning in ways that defied logic. “I’ll do my best,” he said, though his heart felt uneasy.

For three days and nights, Elias and Lena worked on the watch. They discovered that it responded to emotions; anger sped up its ticking, while calmness slowed it. When they finally repaired it, the stranger returned, as if summoned by the watch’s completion.

“Thank you,” he said, slipping the watch into his pocket. He handed Elias a small, glowing vial. “Drink this, and you’ll understand time as I do.”

Elias hesitated. He didn’t want to understand time—he wanted to cherish it. But Lena, curious and eager, reached for the vial. Before Elias could stop her, she drank.

For a moment, nothing happened. Then Lena gasped. Her eyes turned silvery, reflecting endless spirals of time. “I can see it all,” she whispered. “The past, the future. It’s beautiful.”

The stranger smiled darkly and vanished into the night.

Lena’s newfound gift was both a blessing and a curse. She could foresee moments of joy, sorrow, and even danger, but her visions became overwhelming. She started to distance herself from the village, her mind lost in the endless flow of time.

Elias, heartbroken, worked tirelessly to create something that could restore Lena’s balance. Drawing inspiration from his masterpiece clock, he built a delicate pendant with a single dial. When worn, it would let Lena focus on the present, shielding her from the chaos of time’s currents.

When he gave Lena the pendant, tears welled in her eyes. “You’ve saved me,” she said, her voice trembling. “But I can’t stay here. The clock’s gift changed me. I must travel and help others understand time’s value.”

Elias understood. Though it pained him to say goodbye, he knew Lena’s journey was her own. She left Everturn, carrying the pendant and her unique gift into the world.

Years later, when Elias was old and gray, a young woman arrived at his shop, holding a pendant identical to the one he had made for Lena. “My name is Mira,” she said. “Lena was my mentor. She told me about you.”

As Elias listened to Mira’s stories, he smiled, knowing that his apprentice’s legacy lived on. The clock on his wall chimed softly, as if in agreement.

And so, in the quiet village of Everturn, the clockmaker’s gift continued to shape the lives of those who understood the true value of time."""
    
pf = PromptFunction(prompt)
response_format={
                    "name": "story_dict",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "content": {
                                "name": "name of the character",
                                "Occupation":"Describe the character's occupation. Example: 'Student'",
                                "gender": "male or female",
                                "Age": "estimate age of the character",
                                "Outfit_description": {
                                    "top": {
                                        "description": "Describe the upper body outfit such as shirt, hat, glasses etc.",
                                        "type": "string"
                                    },
                                    "bottom": {
                                        "description": "describe the lower body outfit such as shoes, skirt, jeans, etc.",
                                        "type": "string"
                                    },
                                    "accessory": {
                                        "description": "descripe any accessroy character may have.",
                                        "type": "string"
                                    }
                                },
                                "required": ["title", "author", "content"],
                                "additionalProperties": False 
                            }
                        },
                        "required": ["content", "next_page"],
                        "additionalProperties": False
                    }
}
pf.set_temperature(0.1)
response = pf.execute(story = story, RESPONSE_FORMAT=response_format)

this below is for dalle_3 descriptions

In [20]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://img1.getimg.ai/generated/46fd4d05-b2d3-4a7b-a134-be4809c60f9b/img-gF2dbWBI6VkJXgFQPR0Ob.jpeg",
                    }
                },
            ],
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The image depicts a man and a woman in a dimly lit, natural setting, possibly a cave or forest area. They are focused on a glowing bottle that they are holding together, which casts light on their faces and nearby objects. The scene is atmospheric and mystical, with additional lights resembling lanterns hanging in the background.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [22]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a good story teller."},
        {
            "role": "user",
            "content": """base on the input generate me a educational story,image_1:The image shows an older man with white hair and a beard, wearing a work apron, sitting at a workbench. He is focused on adjusting or inspecting an intricate, vintage-looking mechanical device, surrounded by various tools and components. The setting appears to be a workshop or a lab, with a well-organized and cluttered atmosphere, suggesting a focus on craftsmanship or scientific work.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None
image_2:ChatCompletionMessage(content='The image shows an elderly man with a white beard standing in a picturesque setting. He appears to be in a stone archway or doorway. The background is a scenic landscape with mountains, a lake, and some houses. The man is wearing a green vest, a shirt, and has a tool belt. The setting suggests a peaceful and rural environment, possibly historical or fantasy-themed.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
image_3:ChatCompletionMessage(content='The image depicts a man and a woman in a dimly lit, natural setting, possibly a cave or forest area. They are focused on a glowing bottle that they are holding together, which casts light on their faces and nearby objects. The scene is atmospheric and mystical, with additional lights resembling lanterns hanging in the background.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
"""
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Once upon a time, in a quaint village nestled between emerald hills and shimmering lakes, there lived an elderly craftsman named Gideon. With his white hair and beard like the first snow of winter, Gideon was renowned for his skills in creating intricate mechanical devices. His workshop was a realm of wonder, filled with tools, gears, and mysterious components, all neatly organized despite the seeming clutter.\n\nEvery morning, Gideon donned his work apron and busied himself at his workbench, in the heart of his workshop. He would meticulously adjust and inspect the vintage contraptions he loved to build. His most recent project was a clockwork device rumored to unlock the secrets of the stars.\n\nA short distance away from the village center, beyond a stone archway that framed a breathtaking view of the mountains, Gideon often took respite. Wearing his green vest and tool belt, he would stroll through the peaceful landscape, drawing inspiration from the 

In [24]:
prompt = "dragon enjoy sunbath on beach"